# MQTT python 라이브러리 설치

In [1]:
%pip install paho-mqtt


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# MQTT_Subscribe

In [4]:
import time
import paho.mqtt.client as paho

broker="172.30.1.7"

#define callback
def on_message(client, userdata, message):
    time.sleep(1)
    recvData = str(message.payload.decode("utf-8"))
    print("received message =", recvData)

client = paho.Client()
client.on_message=on_message

print("connecting to broker ",broker)
client.connect(broker)#connect
client.loop_start() #start loop to process received messages
print("subscribing ")
client.subscribe("Sensor")#subscribe 토픽이름 설정
time.sleep(5)
client.disconnect() #disconnect
client.loop_stop() #stop loop
time.sleep(10)

connecting to broker  172.30.1.7
subscribing 
received message = {"temp": 23.0,"humi": 38.1}



# MQTT_Subscribe_JSON

In [6]:
import time
import paho.mqtt.client as paho
import json

HOST="172.30.1.7"
recvData = ""

#define callback
def on_message(client, userdata, message):
    time.sleep(1)
    recvData = str(message.payload.decode("utf-8"))
    print("received message =", recvData)
    jsonData = json.loads(recvData) #json 데이터를 dict형으로 파싱

    print(f"Temp: {jsonData['Temp']}")
    print(f"Humi: {jsonData['Humi']}")

client = paho.Client()
client.on_message=on_message

print("connecting to broker ",broker)
client.connect(HOST)#connect

print("subscribing ")
client.subscribe("Sensor")#subscribe

try:
    client.loop_forever()
except KeyboardInterrupt:
    client.disconnect()
    print("disconnect.")


connecting to broker  172.30.1.7
subscribing 
received message = {"Temp": 23.0,"Humi": 38.1}

Temp: 23.0
Humi: 38.1
received message = {"Temp": 23.0,"Humi": 38.1}

Temp: 23.0
Humi: 38.1
received message = {"Temp": 23.0,"Humi": 38.1}

Temp: 23.0
Humi: 38.1
disconnect.


# Python JSON 만들기

In [7]:
import json
import time
import random
from collections import OrderedDict

data = OrderedDict()

data['model'] = "SmartPrediction"
data['time'] = time.strftime('%Y%m%d %H:%M:%S',time.localtime())
data['predict'] = random.randint(0,1)

jStr = json.dumps(data)
jStr

'{"model": "SmartPrediction", "time": "20221012 16:12:19", "predict": 1}'

# MQTT_Publish

In [8]:
import time
import random
import json
import paho.mqtt.client as paho


HOST="172.30.1.7"

def makeJSON():
    data = OrderedDict()

    data['model'] = "SmartPrediction"
    data['time'] = time.strftime('%Y%m%d %H:%M:%S',time.localtime())
    data['predict'] = random.randint(0,1)
    result = json.dumps(data)
    print("predict message = ", result)
    return result


#define callback
def on_message(client, userdata, message):
    time.sleep(1)    

client = paho.Client()
client.on_message=on_message

print("connecting to broker ",broker)
client.connect(HOST)#connect

client.publish('Model',makeJSON()) #publish
try:
    client.loop_forever()
except KeyboardInterrupt:
    print("disconnect.")


connecting to broker  172.30.1.7
predict message =  {"model": "SmartPrediction", "time": "20221012 16:18:15", "predict": 1}
disconnect.


# MQTT_Subscribe_Model_Publish

In [10]:
import time
import paho.mqtt.client as paho
import json

HOST="172.30.1.57"

def predictOfModel(temp, humi): #전처리부분이 이쪽이다.
    pred = random.randint(0,1)
    print("predict =", pred)

    return pred

def makeJSON(pred):
    data = OrderedDict()

    data['model'] = "SmartPrediction"
    data['time'] = time.strftime('%Y%m%d %H:%M:%S',time.localtime())
    data['predict'] = pred
    result = json.dumps(data)
    print("predict message = ", result)
    return result

def readJSON(message):
    recvData = str(message.payload.decode("utf-8"))
    print("received message =", recvData)
    jsonData = json.loads(recvData) #json 데이터를 dict형으로 파싱
    # print(f"Temp: {jsonData['Temp']}")
    # print(f"Humi: {jsonData['Humi']}")

    return jsonData['Temp'],jsonData['Humi']

#define callback
def on_message(client, userdata, message):
    time.sleep(1)
    
    temp,humi = readJSON(message)    #NodeMCU로부터 받은 센서 메시지 파싱
    pred = predictOfModel(temp,humi) #머신러닝/딥러닝 모델 결과
    preJSON = makeJSON(pred)         #NodeMCU로 보낼 메시지 만들기  

    client.publish('Model',preJSON) #publish

client = paho.Client()
client.on_message=on_message

print("connecting to broker ",broker)
client.connect(HOST)#connect

print("subscribing ")
client.subscribe("Sensor")#subscribe

try:
    client.loop_forever()
except KeyboardInterrupt:
    print("disconnect.")


connecting to broker  172.30.1.7
subscribing 
received message = {"Temp":24.5,"Humi":42}
predict = 0
predict message =  {"model": "SmartPrediction", "time": "20221012 16:36:33", "predict": 0}
received message = {"Temp":24.5,"Humi":42}
predict = 0
predict message =  {"model": "SmartPrediction", "time": "20221012 16:36:35", "predict": 0}
received message = {"Temp":24.5,"Humi":42}
predict = 1
predict message =  {"model": "SmartPrediction", "time": "20221012 16:36:37", "predict": 1}
received message = {"Temp":24.5,"Humi":42}
predict = 1
predict message =  {"model": "SmartPrediction", "time": "20221012 16:36:39", "predict": 1}
received message = {"Temp":24.5,"Humi":42}
predict = 1
predict message =  {"model": "SmartPrediction", "time": "20221012 16:36:41", "predict": 1}
received message = {"Temp":24.5,"Humi":42}
predict = 0
predict message =  {"model": "SmartPrediction", "time": "20221012 16:36:43", "predict": 0}
received message = {"Temp":24.5,"Humi":42}
predict = 0
predict message =  {"mo

In [4]:
import paho.mqtt.client as mqtt


def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("connected OK")
    else:
        print("Bad connection Returned code=", rc)


def on_disconnect(client, userdata, flags, rc=0):
    print(str(rc))


def on_subscribe(client, userdata, mid, granted_qos):
    print("subscribed: " + str(mid) + " " + str(granted_qos))


def on_message(client, userdata, msg):
    print(str(msg.payload.decode("utf-8")))


# 새로운 클라이언트 생성
client = mqtt.Client()
# 콜백 함수 설정 on_connect(브로커에 접속), on_disconnect(브로커에 접속중료), on_subscribe(topic 구독),
# on_message(발행된 메세지가 들어왔을 때)
client.on_connect = on_connect
client.on_disconnect = on_disconnect
client.on_subscribe = on_subscribe
client.on_message = on_message
# 로컬 아닌, 원격 mqtt broker에 연결
# address : broker.hivemq.com
# port: 1883 에 연결
# client.connect('broker.hivemq.com', 1883)
client.connect('localhost', 1883)
# test/hello 라는 topic 구독
client.subscribe('Sensor')
client.loop_forever()

connected OK
subscribed: 1 (1,)
asas


KeyboardInterrupt: 